In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#@title DataLoad [gdown]
import gdown, zipfile, os

file_id = '1hZEUQJqxv60MmVNGigf_ig9mdkQFPLIm'
output = 'file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'cloud'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(output, 'r') as z:
  z.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1hZEUQJqxv60MmVNGigf_ig9mdkQFPLIm
From (redirected): https://drive.google.com/uc?id=1hZEUQJqxv60MmVNGigf_ig9mdkQFPLIm&confirm=t&uuid=06b6d85f-06db-4c4a-b7b6-b2a67b54bce4
To: /content/file.zip
100%|██████████| 187M/187M [00:06<00:00, 26.9MB/s]


In [ ]:
#@title DataLoad (cloud/data/train)
from sklearn.model_selection import train_test_split
def load_data(base_dir='cloud'):
  images_path = []
  labels = []

  class_names = []

  for class_name in os.listdir(base_dir):
    class_name_path = os.path.join(base_dir, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        images_path.append(image_path)
        labels.append(class_name)

  return np.array(images_path), np.array(labels)

images_path, labels = load_data()
print(images_path.shape, labels.shape)
print(np.unique(labels))

(13104,) (13104,)
['Ac' 'As' 'Cb' 'Cc' 'Ci' 'Cs' 'Ct' 'Cu' 'Ns' 'Sc' 'St']


In [1]:
#@title train_validation data split
from sklearn.model_selection  import train_test_split

train_images_path ,test_image_path, train_labels, test_labels=train_test_split(images_path, labels, test_size=0.1, random_state=42)
train_images_path ,val_images_path, train_labels, val_labels=train_test_split(train_images_path, train_labels, test_size=0.2, random_state=42)
train_images_path.shape, val_images_path.shape, train_labels.shape, val_labels.shape,test_image_path.shape, test_labels.shape

NameError: name 'images_path' is not defined

In [ ]:
#@title label에 대해 label_encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
train_labels = np.array(train_labels, dtype='int32')

val_labels = label_encoder.transform(val_labels)
val_labels = np.array(val_labels, dtype='int32')


test_labels = label_encoder.transform(test_labels)
test_labels = np.array(test_labels, dtype='int32')

NameError: name 'train_labels' is not defined

In [ ]:
#@title EfficientNet model 전이학습
from sklearn.model_selection import StratifiedShuffleSplit
from skimage.transform import resize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0


height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
efnb0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape, classes=2)

model = Sequential()
model.add(efnb0)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X, y, epochs = 30)